# MultiLabel Classification

### Dependencies:
#### Python
* pip install sklearn
* pip install scikit-multilearn
* pip install future
* pip install python-igraph
* python-graph-tool: Use this tutorial to install it on ubuntu https://zhangkaiyuan.com/2018/03/10/Install-graphtools-on-Ubuntu/

### Sources: (should be at the bottom)
* http://scikit.ml/api/classify.html#ensemble-approaches
* http://scikit.ml/
* https://en.wikipedia.org/wiki/Multi-label_classification
* https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/

## Generate data

In [19]:
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

X, y = make_multilabel_classification(sparse=True, n_labels=20, return_indicator = 'sparse', allow_unlabeled = False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Techniques:

## 1. Transformation methods:
We will transform multi-label classification problem into single lable

### 1.1 Binary relevance:
This baseline approach, amounts to independently training one binary classifier for each label: Given an unseen sample, the combined model then predicts all labels for this sample for which the respective classifiers predict a positive result

In [20]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


classifier = BinaryRelevance(GaussianNB())

# train:
classifier.fit(X_train, y_train)

# predict:
prediction = classifier.predict(X_test)

# check accuracy:
accuracy_score(y_test, prediction)

0.65000000000000002

### 1.2 Classifier chains:
In this method, each classifier is trained on the output of the previous classifier (when the first one is trained on the input data)

In [21]:
from skmultilearn.problem_transform import ClassifierChain

classifier = ClassifierChain(GaussianNB())

# train:
classifier.fit(X_train, y_train)

# predict:
prediction = classifier.predict(X_test)

# check accuracy:
accuracy_score(y_test, prediction)

0.5

### 1.3 Label powerset:
This transformation creates one binary classifier for every label combination found in the training set.

In [22]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(GaussianNB())

# train:
classifier.fit(X_train, y_train)

# predict:
prediction = classifier.predict(X_test)

# check accuracy:
accuracy_score(y_test, prediction)

0.75

## 2. Adapted algorithm
Instead of changing the problems, we could change the algorithm to support multi label classification. Exmaple of these algorithms are KNN, desicion trees, boosting, neural networks, ets 

### 2.1 Multi Lavel KNN (MLkNN)

In [23]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=20)

# train
classifier.fit(X_train, y_train)

# predict
prediction = classifier.predict(X_test)

# check accuracy
accuracy_score(y_test, prediction)

0.69999999999999996

## 3. Ensemble learning
It is often useful to train more than one model for a subset of labels in multi-label classification, especially for large label spaces - a well-selected smaller label subspace can allow more efficient classification.
As rule of thumb, ensemble methods use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms alone

In [24]:
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.cluster import IGraphLabelCooccurenceClusterer
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

# construct base forest classifier
base_classifier = RandomForestClassifier()

# setup problem transformation approach with sparse matrices for random forest
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

# partition the label space using fastgreedy community detection
# on a weighted label co-occurrence graph with self-loops allowed
clusterer = IGraphLabelCooccurenceClusterer('fastgreedy', weighted=True,
    include_self_edges=True)

# setup the ensemble metaclassifier
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

accuracy_score(y_test, prediction)

0.69999999999999996